In [ ]:
! pip install -Uq transformers sentence_transformers datasets

In [ ]:
import wandb
wandb.init(mode="disabled")

### **Load Dataset**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yosefw/amharic-news-retrieval-dataset-v2-with-negatives-V2")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 61469
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 6832
    })
})

In [ ]:
# rename columns
dataset = dataset.rename_column("query", "anchor")
dataset = dataset.rename_column("passage", "positive")
dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 61469
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 6832
    })
})

In [ ]:
from datasets import Dataset
from tqdm import tqdm
import random

ds_rows = []
for row in tqdm(dataset["train"]):
  neg_passages = row["negative_passages"]
  # neg_passages = list(filter(lambda x: x["passage_id"] not in test_passage_ids, neg_passages))
  neg_passages_filtered = neg_passages[:2] + neg_passages[-2:]

  for neg_passage in neg_passages_filtered:
    ds_rows.append({
        "query_id": row["query_id"],
        "passage_id": row["passage_id"],
        "anchor": row["anchor"],
        "positive": row["positive"],
        "negative": neg_passage["passage"],
      })

relevance_dataset = Dataset.from_list(ds_rows).shuffle(seed=42)#.sort("query_id")#.select(range(4000))
relevance_dataset

100%|██████████| 61469/61469 [00:26<00:00, 2307.96it/s]


Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'negative'],
    num_rows: 245876
})

### **Initialize SPLADE Model**

In [ ]:
from sentence_transformers import SparseEncoder, SparseEncoderModelCardData

# 1. Load a model to finetune with 2. (Optional) model card data
model = SparseEncoder(
    "rasyosef/roberta-base-amharic",
    model_card_data=SparseEncoderModelCardData(
        language="am",
        license="mit",
        model_name="SPLADE-RoBERTa-Amharic-Base",
    )
)

In [ ]:
from sentence_transformers.sparse_encoder.losses import SparseMultipleNegativesRankingLoss, SpladeLoss
from sentence_transformers.sparse_encoder.evaluation import SparseInformationRetrievalEvaluator

# 4. Define a loss function
loss = SpladeLoss(
    model=model,
    loss=SparseMultipleNegativesRankingLoss(model=model),
    query_regularizer_weight=2e-3,
    document_regularizer_weight=1e-3,
)

### **Evaluator**

In [ ]:
from datasets import concatenate_datasets

train_dataset = dataset["train"]
test_dataset = dataset["test"]
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])

corpus_dataset

Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'category', 'link', 'source_dataset', 'negative_passages'],
    num_rows: 68301
})

In [ ]:
# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset["passage_id"], corpus_dataset["positive"])
) # Our corpus (cid => document)
queries = dict(
    zip(test_dataset["query_id"], test_dataset["anchor"])
) # Our queries (qid => question)

In [ ]:
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}
for row in test_dataset:
  relevant_docs[row["query_id"]] = [row["passage_id"]]

In [ ]:
evaluator = SparseInformationRetrievalEvaluator(
    queries=queries,
    corpus=corpus,
    relevant_docs=relevant_docs,
    batch_size=128,
    corpus_chunk_size=2048,
    show_progress_bar=False
)

In [ ]:
# evaluator(model)

### **Train**

In [ ]:
from sentence_transformers import SparseEncoderTrainer, SparseEncoderTrainingArguments
from sentence_transformers.training_args import BatchSamplers

num_epochs = 6
batch_size = 32
gradient_accum_steps = 2

# 5. (Optional) Specify training arguments
run_name = "SPLADE-RoBERTa-Amharic-Base-V4"
args = SparseEncoderTrainingArguments(
    # Required parameter:
    output_dir=f"models/{run_name}",
    # Optional training parameters:
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accum_steps,
    per_device_eval_batch_size=batch_size,
    learning_rate=6e-5,
    warmup_ratio=0.025,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    run_name=run_name,  # Will be used in W&B if `wandb` is installed
)

# 7. Create a trainer
trainer = SparseEncoderTrainer(
    model=model,
    args=args,
    train_dataset=relevance_dataset.select_columns(['anchor', 'positive', 'negative']),
    loss=loss,
    evaluator=evaluator,
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [ ]:
# Train
trainer.train()

Epoch,Training Loss,Validation Loss,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100,Query Active Dims,Query Sparsity Ratio,Corpus Active Dims,Corpus Sparsity Ratio,Regularizer Weight
1,35.024500,No log,0.604716,0.783392,0.828940,0.880199,0.604716,0.261131,0.165788,0.088020,0.604716,0.783392,0.828940,0.880199,0.745644,0.702171,0.705743,331.084076,0.989654,1119.335265,0.965021,0.000500
2,0.061400,No log,0.618043,0.791301,0.838313,0.884153,0.618043,0.263767,0.167663,0.088415,0.618043,0.791301,0.838313,0.884153,0.755098,0.713278,0.717069,138.318390,0.995678,346.732935,0.989165,0.001999
3,0.032200,No log,0.652753,0.819713,0.861453,0.902460,0.652753,0.273238,0.172291,0.090246,0.652753,0.819713,0.861453,0.902460,0.782751,0.743820,0.746809,113.116875,0.996465,295.223656,0.990774,0.002000
4,0.016900,No log,0.661687,0.826889,0.868190,0.907293,0.661687,0.275630,0.173638,0.090729,0.661687,0.826889,0.868190,0.907293,0.790377,0.752285,0.755025,86.881516,0.997285,220.327823,0.993115,0.002000
5,0.009800,No log,0.666374,0.830844,0.869801,0.907147,0.666374,0.276948,0.173960,0.090715,0.666374,0.830844,0.869801,0.907147,0.792930,0.755664,0.758562,72.095490,0.997747,165.225698,0.994837,0.002000
6,0.008000,No log,0.663152,0.832015,0.871119,0.906268,0.663152,0.277338,0.174224,0.090627,0.663152,0.832015,0.871119,0.906268,0.791667,0.754148,0.757056,69.623611,0.997824,153.635892,0.995199,0.002000


TrainOutput(global_step=23052, training_loss=5.858804887933448, metrics={'train_runtime': 19153.6612, 'train_samples_per_second': 77.022, 'train_steps_per_second': 1.204, 'total_flos': 0.0, 'train_loss': 5.858804887933448, 'epoch': 6.0, 'document_regularizer_weight': 0.001, 'query_regularizer_weight': 0.002})

In [ ]:
# 8. Evaluate the model performance again after training
evaluator(model)

{'dot_accuracy@1': 0.6631517281780902,
 'dot_accuracy@3': 0.8320152314001171,
 'dot_accuracy@5': 0.8711189220855302,
 'dot_accuracy@10': 0.9062683069712947,
 'dot_precision@1': 0.6631517281780902,
 'dot_precision@3': 0.2773384104667057,
 'dot_precision@5': 0.174223784417106,
 'dot_precision@10': 0.09062683069712946,
 'dot_recall@1': 0.6631517281780902,
 'dot_recall@3': 0.8320152314001171,
 'dot_recall@5': 0.8711189220855302,
 'dot_recall@10': 0.9062683069712947,
 'dot_ndcg@10': 0.7916669501248136,
 'dot_mrr@10': 0.7541483596953714,
 'dot_map@100': 0.7570560398560239,
 'query_active_dims': 69.62361145019531,
 'query_sparsity_ratio': 0.9978242621421815,
 'corpus_active_dims': 153.6358920497414,
 'corpus_sparsity_ratio': 0.9951988783734458}

In [ ]:
# 9. Save the trained model
model.save_pretrained(f"./final")

In [ ]:
# 10. (Optional) Push it to the Hugging Face Hub

import os
from google.colab import userdata

os.environ["HF_TOKEN"] = userdata.get("HF_WRITE_YOSEFW")

# # push model to hub
trainer.model.push_to_hub(run_name, exist_ok=True, private=True)

### **Testing**

In [ ]:
model

SparseEncoder(
  (0): MLMTransformer({'max_seq_length': 510, 'do_lower_case': False, 'architecture': 'XLMRobertaForMaskedLM'})
  (1): SpladePooling({'pooling_strategy': 'max', 'activation_function': 'relu', 'word_embedding_dimension': 32000})
)

In [ ]:
sentences = [
    "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
    "በማዕከላዊ ኢትዮጵያ ክልል ሃድያ ዞን ጊቤ ወረዳ በሚገኙ 12 ቀበሌዎች መሠረታዊ የመንግሥት አገልግሎት መስጫ ተቋማት በሙሉና በከፊል በመዘጋታቸው መቸገራቸውን ነዋሪዎች አመለከቱ። ከባለፈው ዓመት ጀምሮ የጤና፣ የትምህርት እና የግብር አሰባሰብ ሥራዎች በየአካባቢያቸው እየተከናወኑ አለመሆናቸውንም ለዶቼ ቬለ ተናግረዋል።",
    "የሕዝብ ተወካዮች ምክር ቤት አባል እና የቋሚ ኮሚቴ ሰብሳቢ የነበሩት አቶ ክርስቲያን ታደለ እና የአማራ ክልል ምክር ቤት አባል የሆኑት አቶ ዮሐንስ ቧያለው ከቃሊቲ ወደ ቂሊንጦ ማረሚያ ቤት መዛወራቸውን ጠበቃቸው ተናገሩ።",
    "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡"
  ]

embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities)

decoded = model.decode(embeddings, top_k=16)
for decoded, sentence in zip(decoded, sentences):
    print(f"Sentence: {sentence}")
    print(f"Decoded: {decoded}")
    print()

tensor([[4.3887e+01, 1.0661e-03, 0.0000e+00, 3.0628e+01],
        [1.0661e-03, 5.1914e+01, 4.3003e+00, 2.1406e+00],
        [0.0000e+00, 4.3003e+00, 3.8541e+01, 0.0000e+00],
        [3.0628e+01, 2.1406e+00, 0.0000e+00, 6.3737e+01]], device='cuda:0')
Sentence: የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል
Decoded: [('▁ገሞራ', 1.8486328125), ('▁በአፋር', 1.77734375), ('▁የመሬት', 1.74609375), ('▁የአፋር', 1.69140625), ('▁አፋር', 1.615234375), ('▁መንቀጥቀጥ', 1.59765625), ('አፋሮች', 1.462890625), ('▁እሳተ', 1.4365234375), ('ሳተ', 1.42578125), ('▁ክልል', 1.23828125), ('▁መሬቱ', 1.2099609375), ('▁ምልክት', 1.1259765625), ('ፋር', 1.1240234375), ('▁በመሬት', 1.087890625), ('▁መንሸራተት', 1.0810546875), ('ተደጋገመ', 0.9228515625)]

Sentence: በማዕከላዊ ኢትዮጵያ ክልል ሃድያ ዞን ጊቤ ወረዳ በሚገኙ 12 ቀበሌዎች መሠረታዊ የመንግሥት አገልግሎት መስጫ ተቋማት በሙሉና በከፊል በመዘጋታቸው መቸገራቸውን ነዋሪዎች አመለከቱ። ከባለፈው ዓመት ጀምሮ የጤና፣ የትምህርት እና የግብር አሰባሰብ ሥራዎች በየአካባቢያቸው እየተከናወኑ አለመሆናቸውንም ለዶቼ ቬለ ተናግረዋል።
Decoded: [('▁ጊቤ', 1.87890625), ('▁በማዕከላዊ', 1.36328125), ('▁መስጫ', 1.318359375), ('ዘጉ', 1.274414062

In [ ]:
stats = SparseEncoder.sparsity(embeddings)
print(f"Sparsity: {stats['sparsity_ratio']:.2%}")  # Typically >99% zeros
print(f"Avg non-zero dimensions per embedding: {stats['active_dims']:.2f}")

Sparsity: 99.66%
Avg non-zero dimensions per embedding: 108.75
